In [ ]:
import functools
import inspect


def build_config(init_func):
    @functools.wraps(init_func)
    def wrapper(self, *args, **kwargs):
        sig = inspect.signature(init_func)
        param_pairs = list(sig.parameters.items())[1:]
        
        config = {
            name: None if param.default is inspect.Parameter.empty else param.default
            for name, param in param_pairs
        }

        config.update(dict(zip(config.keys(), args)))
        config.update(kwargs)
        self.config = config

        init_func(self, *args, **kwargs)

    return wrapper


In [ ]:
import torch

x = torch.randn(16, 2)


In [ ]:
class MyClass:
    @build_config
    def __init__(
        self, 
        name, 
        surname, 
        age,
        weight, 
        height,
        sex='fmj',
        race='xxxxxx', 
        features=None,
        aux_stuff=None,
    ):
        self.here = 'there'
    
    def add(self, a, b):
        print(self.here)
        return a + b


In [82]:
cls = MyClass('peter', 'borrow', 12, 124, 0.234, features=(1, 2, 'bdick'))
print('-----')
cls.config

-----


{'name': 'peter',
 'surname': 'borrow',
 'age': 12,
 'weight': 124,
 'height': 0.234,
 'sex': 'fmj',
 'race': 'xxxxxx',
 'features': (1, 2, 'bdick'),
 'aux_stuff': None}

In [22]:
import inspect

def collect_args_and_kwargs(cls):
    """
    A class decorator to collect all __init__ arguments (with their names and values)
    into a dictionary and store it as an instance attribute.
    """
    original_init = cls.__init__

    def new_init(self, *args, **kwargs):
        # Get the signature of the __init__ method
        sig = inspect.signature(original_init)
        bound_args = sig.bind(self, *args, **kwargs)
        bound_args.apply_defaults()

        # Exclude 'self' and store the remaining arguments in a dictionary
        self.init_args = {key: value for key, value in bound_args.arguments.items() if key != 'self'}

        # Call the original __init__ method
        original_init(self, *args, **kwargs)

    cls.__init__ = new_init
    return cls

# Example usage
@collect_args_and_kwargs
class ExampleClass:
    def __init__(self, a, b, c=3, d=4):
        self.a = a
        self.b = b
        self.c = c
        self.d = d

# Instantiate the class
example = ExampleClass(1, 2, d=5)

# Access the collected arguments with their names and values
print(example.init_args)


{'a': 1, 'b': 2, 'c': 3, 'd': 5}
